# The CKY Algorithm and Syntactic Parsing

### Learning Goals

Today's activity will be to implement CKY parsing 

#### Getting Started

As you saw in the reading, in order to apply a parsing technique like CKY, we need to have grammars in a very particular form: *Chomsky-Normal Form*, or *CNF*. CNF Grammars are constrained insofar as they can only contain two kinds of rules:

1. Rules of the form $A \rightarrow B C$, which I will call *Grammar Rules*, which allow a nonterminal $A \in N$ to expand to two nonterminals $B, C \in N$.
2. Rules of the form $A \rightarrow w$, which I will call *Lexical Rules*, which allow a nonterminal $A \in N$ to expand to a single terminal $w \in \Sigma$.

Now, for the purposes of time, I won't ask you to write an algorithm to convert any CFG to CNF (though you can do that, and J&M explain the key ideas in 18.6.1, which I'll expect you to know!) However, I will introduce a new Grammar class that enforces the CNF constraints, `CNFGrammar`. 

To understand how it's structured, I'd like you to write two methods for the class:
1. Write `merges_to`, which, given $B, C \in N$, returns a list of nonterminals that can expand to $B$ $C$ under the grammar (i.e., through a grammar rule). 
2. Write `tag`, which, given $w \in \Sigma$, returns a list of nonterminals that can expand to $w$ (i.e., through a lexical rule).

**Make sure you understand how the grammar is structured before moving on**.

Note that the representation of the grammar is not optimized for `merges_to` and `tag` to run as efficiently as they could. As a **bonus** you can restructure how `LR` and `GR` are structured to do that optimization, but do that after completing everything else: We're just seeking the standard CKY asymptotic complexity of $O(n^3)$, with $n$ being the length of the sentence. 

In [ ]:
from typing import Iterable, Sequence, Mapping, Tuple
import numpy as np
import random

class CNFGrammar:
    def __init__(self,
                 S : str, 
                 T : Iterable[str], 
                 NT : Iterable[str], 
                 LR : Mapping[str, Sequence[str]],
                 GR : Mapping[str, Sequence[Sequence[str]]]):
        self.S = S
        self.T = T
        self.NT = NT
        self.LR = LR
        self.GR = GR

    def get_start(self) -> str:
        return self.S
        
    def print_rules(self) -> None:
        print("---Lexical Rules")
        for lhs, rhss in self.LR.items():
            print("{} -> {}".format(lhs, ", ".join(rhss)))
        print("---Grammar Rules")
        for lhs, rhss in self.GR.items():
            for rhs in rhss:
                print("{} -> {}".format(lhs, " ".join(rhs)))

    def merges_to(self, B : str, C : str) -> Iterable[str]:
        # TODO: return a list of all NTs A such that 
        # A -> B C is in the grammar
        out = []
        return out

    def tag(self, a : str) -> Sequence[str]:
        # TODO: return a list of all NTs A such that
        # A -> a is in the grammar
        out = []
        return out


Below is the CNF Grammar from Figure 18.10 from the J&M. Use the next two cells to confirm that your implementations are working!

In [ ]:
# From the J&M Reading

S = "S"
T = set(["book", "flight", "meal", "money", "include",
     "prefer", "does",
     "me", "I", "she",
     "Houston", "NWA",
     "the", "a", "this", "that",
     "from", "to", "on", "near", "through"])
NT = set(["N", "V", "X1", "X2", "D", "P", "Aux", "S", "NP", "Nom", "VP", "PP"])
LR = {
        # Lexical Rules
        "N": ["book", "flight", "meal", "money"],
        "V": ["book", "include", "prefer"],
        "D": ["the", "a", "that", "this"],
        "P": ["from", "to", "on", "near", "through"],
        "Conj": ["and", "or", "but"],
        "Aux": ["does"],
        "NP":["me", "she", "I", "Houston", "NWA"],
        "Nom":["book", "flight", "meal", "money"],
        "VP":["book", "include", "prefer"],
        "S":["book", "include", "prefer"]
}
GR = {
        # Grammar Rules
        "S":[["NP", "VP"],
             ["X1", "VP"],
             ["V", "NP"],
             ["X2", "PP"],
             ["V", "PP"],
             ["VP", "PP"]],
        "X1":[["Aux", "NP"]],
        "NP":[["D", "Nom"]],
        "Nom":[["Nom", "N"], 
               ["Nom", "PP"]],
        "VP":[["V", "NP"], 
              ["X2", "PP"],
              ["V", "PP"]],
        "X2":[["V", "NP"]],
        "PP":[["P", "NP"]]
}

JMGrammar = CNFGrammar(S, T, NT, LR, GR)
JMGrammar.print_rules()

In [ ]:
print(JMGrammar.merges_to("D", "Nom")) # Should be [NP]
print(JMGrammar.tag("book")) # Should be [N, V, Nom, VP, S]

#### Implementing CKY Recognition

Now time to implement the CKY algorithm yourself! 

Remember that we...
1. Tag each word with possible nonterminals using lexical rules and store the results along the main diagonal of our chart.
2. For each size of constituent/*span*, from smallest to largest...
    1. For each span of that size...
       1. For each way to split that span into two halves...
          1. We check the grammar to see whether the two halves of the split can merge into a larger nonterminal via a Grammar Rule (check the corresponding cells for those smaller halves to see what nonterminals they can be!). We add these to the cell corresponding to the full constituent.
3. You have one cell in the chart corresponding to the full sentence --- which one?) If the start symbol is contained in that cell, the sentence is grammatical!

I recommend implementing your code so that it mirrors how you'd implement the algorithm by hand! If you *don't* know how you'd implement it by hand, I would recommend stepping through the example from the reading with your partner!

In [ ]:
def printChart(chart : Iterable[Iterable[Iterable[str]]]) -> None:
    # May be useful for debugging your chart
    for i in range(len(chart)):
        print("\t".join([str(x).ljust(20) for x in chart[i]]))

def CKYRecognizer(grammar : CNFGrammar, sent : Sequence[str]) -> bool:
    # TODO: Implement the CKY algorithm
    return False

In [ ]:
CKYRecognizer(JMGrammar, "book the flight through Houston".split()) # Should be grammatical

In [ ]:
CKYRecognizer(JMGrammar, "Houston the book flight through".split()) # Should be ungrammatical

#### Implementing Full Parsing

Now, with some additional time, we can do something I find pretty cool: full *parsing*, where we reconstruct the parse tree that would generate the sentence. The way we do this is by adding in some additional book-keeping: Every time we determine a span can form a constituent with a particular non-terminal label (i.e., "through Houston" is a PP), we store not just the non-terminal, but information that lets us know exactly which the two subpieces that got us there. In fact, if we want to be a bit extravagant, we can store the *entire parse tree* that gets us from the string of terminals within the constituent to the nonterminal label. 

In practice, this is a bit unnecessary and terribly inefficient (it's much better to use the textbook's recommendation and store backpointers to elsewhere in the table). Pedagogically (and for the small examples we consider here), it works out a lot more cleanly, so we'll do that here. Though, again, as a **bonus**, you can write a more memory-efficient version!

Here's what you should do:
1. Look at the example code for NLTK's `Tree` class to see how you should can build the relevant tree.
2. Write `CKYParser` below so that it returns a *list* of valid parse trees. If the sentence is ambiguous, you should return *all* trees that represent that ambiguity!
3. Bask in the beautiful parse tree's you've generated!

In [ ]:
from nltk.tree import Tree

B = Tree("D", ["the"])
B

In [ ]:
C = Tree("Nom", ["flight"])
C

In [ ]:
A = Tree("NP", [B, C])
A

In [ ]:
def CKYParser(grammar : CNFGrammar, sent : Sequence[str]) -> Sequence[Tree]:
    # TODO: Update your Recognizer code from above to build parse trees
    return []

If all goes well, you'll get 3 parse trees below! One that begins with $\text{S} \rightarrow \text{V NP}$, another that begins with $\text{S} \rightarrow \text{VP PP}$, and the last that begins with the rule $\text{S} \rightarrow \text{X2 NP}$.

Wow!

In [ ]:
from IPython.display import display

parses = CKYParser(JMGrammar, "book the flight through Houston".split())

for parse in parses:
    display(parse)